# System Analysis

# Initialization

In [ ]:
import pandas as pd
from numpy import log10, sqrt

In [ ]:
def sema(vect, fig):
    """Rewrite the semantics column into a more compact, human readable version.
    """
    nv = []
    for line in vect:
        v = []
        for (x,y) in fig.pepperargs[line].items():
            if x == 'enumconc': continue
            if x == 'max_complex_size': continue
            if x == 'max_complex_count': continue
            if x == 'max_reaction_count': continue
            if x == 'condensed': 
                z = x if y is True else 'detailed'
            else:
                z = '{}={}'.format(x,y)
            v.append(z)
        nv.append(', '.join(v))
    return nv

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

In [ ]:
from zhang2007 import data; z07 = data()
from yin2008 import data; y08 = data()
from zhang2009 import data; z09 = data()
from zhang2010 import data; z10 = data()
from zhang2011 import data; z11 = data()
from kotani2017 import data; k17 = data()

%who list

In [ ]:
def fill_table_prf(data, verb = 0):
    for fig in data:
        #print(fig.name)
        fig.eval(verbose = verb, enumprofile = True)

def fill_table(data, verb = 0):
    for fig in data:
        print(fig.name)
        display(fig.pepperargs['default'])
        assert fig.pepperargs['default']['condensed'] == True
        fig.eval('default', verbose = verb)
        for df in fig.get_dataframes():
            df['Name'] = fig.name
            df['Semantics'] = sema(df['Semantics'], fig)
            yield df

# Peppercorn estimates

In [ ]:
Table1 = pd.DataFrame()
for x in fill_table(z07 + y08 + z09 + z10 + z11 + k17, verb = 0):
    display(x)
    Table1 = Table1.append(x)

In [ ]:
%%timeit
fill_table_prf(z07 + y08 + z09 + z10 + z11 + k17)

# Everything

In [ ]:
Table1.to_latex('ManySystems.tex', index=False, float_format='{:.3g}'.format)
tmpfig = Table1
#tmpfig.reset_index(inplace=True, drop=True)
#tmpfig = tmpfig.iloc[0:-1]
#tmpfig = tmpfig.iloc[[4,5,6,7,28,29,30,31], :]
#display(tmpfig)

if False: # Just look at them individually
    #tmpfig = tmpfig.loc[tmpfig['Name'].isin([fig.name for fig in z07+z10])]
    m = 'o'
else:
    m = ['+'] * 3
    m += ['.']
    m += ['^'] * 3
    m += ['*'] * 4
    m += ['v']
    m += ['o'] * 3

colors = sns.color_palette("Set1", n_colors=8, desat=.5).as_hex()
    
g = sns.lmplot(x = "Time (experiment)", y = "Time (simulation)", 
               col = 'Metric', hue = 'Name', sharey = True, 
               col_order = ['completion-time', 'diagonal-crossing-time'],
               data = tmpfig, markers = m, palette = colors,
               fit_reg = False, legend = False)

plt.xscale('log')
plt.yscale('log')
plt.legend(bbox_to_anchor=(1, -0.2), loc=1, borderaxespad=0.)#, fontsize = 10)

mi, ma = 1e1, 1e5
for ax in g.axes[0]:
    ax.plot([mi, ma], [mi, ma], linewidth=3, color='white', zorder=0)

g.axes[0][0].set_title('50%-completion time')
g.axes[0][1].set_title('Diagonal-crossing time')

g.set(xlabel='Experimental timepoint [s]')  
g.axes[0][0].set(ylabel='Peppercorn timepoint [s]')

plt.savefig('all_systems.pdf', bbox_inches='tight')
plt.savefig('all_systems.svg', bbox_inches='tight')